In [28]:
import jpy_tools.parseSnake as jps

In [29]:
configPath = '/public/home/liuzj/publicPipeline/snuupy/snakemake/config.yaml'
snakePath = '/public/home/liuzj/publicPipeline/snuupy/snakemake/snakefile'

In [30]:
snakeFile = jps.SnakeMakeFile()

In [31]:
snakeHeader = jps.SnakeHeader(snakeFile, configPath)
snakeHeader.generateContent()

config contents:

    pipelineDir     :/public/home/liuzj/publicPipeline/snuupy/

     resultDir      :/public/home/liuzj/publicPipeline/snuupy/example/snakemakeResults/

    geneAnnoBed     :/public/home/liuzj/publicPipeline/snuupy/example/00_data/ref/annoFile/gene.bed

      genomeFa      :/public/home/liuzj/publicPipeline/snuupy/example/00_data/ref/annoFile/genome.fa

     genomeFai      :/public/home/liuzj/publicPipeline/snuupy/example/00_data/ref/annoFile/genome.fa.fai

     picardPath     :/public/home/liuzj/softwares/picard/picard.jar

firstRunCellRangerConfig:/public/home/liuzj/publicPipeline/snuupy/optimizedCellRanger/cellRangerParameter.yaml

secondRunCellRangerConfig:/public/home/liuzj/publicPipeline/snuupy/optimizedCellRanger/rerunCellRangerParameter.yaml

   illuminaFastqs   :/public/home/liuzj/publicPipeline/snuupy/example/00_data/illuminaFastq/



In [32]:
runCellRanger = jps.SnakeRule(snakeFile, 'runCellRanger', 1, 2)
runCellRanger.setInput(b=['firstRunCellRangerConfig'])
runCellRanger.setOutput(a = ['runCellRangerFinished.empty'])
runCellRanger.setShell("""
python ./optimizedCellRanger/optimizedCellRanger.py runCellRanger -p {input.firstRunCellRangerConfig} &&\
touch {output.runCellRangerFinished}
""")
runCellRanger.generateContent()

In [33]:
snakeFile.pathPool

{'step1ResultDir': 'step1ResultDir = f"{config[\'resultDir\']}step1_runCellRanger/"',
 'firstRunCellRangerConfig': "firstRunCellRangerConfig = config['firstRunCellRangerConfig']",
 'runCellRangerFinished': 'runCellRangerFinished = f"{config[\'resultDir\']}step1_runCellRanger/runCellRangerFinished.empty"'}

In [34]:
extractExonBases = jps.SnakeRule(snakeFile, 'extractExonBases', 2, 2)
extractExonBases.setInput(a = ['runCellRangerFinished'])
extractExonBases.setOutput(a = ['extractExonBasesFinished.empty'])
extractExonBases.setParams(a = ['extractExonBasesTemp/', 'extractExonBasesFastq/'], b = ['picardPath', 'geneAnnoBed', 'illuminaFastqs'], c = dict(extractExonBasesCutoff = 30, extractExonBasesInBam = 'f"{config[\'resultDir\']}step1_runCellRanger/test/outs/possorted_genome_bam.bam"'))
extractExonBases.setShell("""
python ./optimizedCellRanger/optimizedCellRanger.py extractExonBases --bam {params.extractExonBasesInBam} --temp {params.extractExonBasesTemp} -t {threads} --picard {params.picardPath} \
    --bed {params.geneAnnoBed} --fastq {params.illuminaFastqs} --out {params.extractExonBasesFastq} --cutoff {params.extractExonBasesCutoff} &&
touch {output.extractExonBasesFinished}
""")
extractExonBases.generateContent()

In [35]:
rerunCellRanger = jps.SnakeRule(snakeFile, 'rerunCellRanger', 3, 2)
rerunCellRanger.setInput(a=['extractExonBasesFinished'], b=['secondRunCellRangerConfig'])
rerunCellRanger.setOutput(a = ['rerunCellRangerFinished.empty'])
rerunCellRanger.setShell("""
python ./optimizedCellRanger/optimizedCellRanger.py runCellRanger -p {input.secondRunCellRangerConfig} &&\
touch {output.rerunCellRangerFinished}
""")
rerunCellRanger.generateContent()

In [36]:
snakeFile.pathPool

{'step1ResultDir': 'step1ResultDir = f"{config[\'resultDir\']}step1_runCellRanger/"',
 'firstRunCellRangerConfig': "firstRunCellRangerConfig = config['firstRunCellRangerConfig']",
 'runCellRangerFinished': 'runCellRangerFinished = f"{config[\'resultDir\']}step1_runCellRanger/runCellRangerFinished.empty"',
 'step2ResultDir': 'step2ResultDir = f"{config[\'resultDir\']}step2_extractExonBases/"',
 'extractExonBasesFinished': 'extractExonBasesFinished = f"{config[\'resultDir\']}step2_extractExonBases/extractExonBasesFinished.empty"',
 'extractExonBasesTemp': 'extractExonBasesTemp = f"{config[\'resultDir\']}step2_extractExonBases/extractExonBasesTemp/"',
 'extractExonBasesFastq': 'extractExonBasesFastq = f"{config[\'resultDir\']}step2_extractExonBases/extractExonBasesFastq/"',
 'picardPath': "picardPath = config['picardPath']",
 'geneAnnoBed': "geneAnnoBed = config['geneAnnoBed']",
 'illuminaFastqs': "illuminaFastqs = config['illuminaFastqs']",
 'extractExonBasesCutoff': 'extractExonBasesCut

In [37]:
parseIllumina = jps.SnakeRule(snakeFile, 'parseIllumina', 4, 2)
parseIllumina.setInput(a = ['rerunCellRangerFinished.empty'])
parseIllumina.setOutput(a = ['parseIlluminaResults.index'])
parseIllumina.setParams(b = ['genomeFai'], c = dict(useBarcodeGz = 'f"{config[\'resultDir\']}step3_rerunCellRanger/test/outs/filtered_feature_bc_matrix/barcodes.tsv.gz"', useBarcode = 'f"{config[\'resultDir\']}step3_rerunCellRanger/test/outs/filtered_feature_bc_matrix/barcodes.tsv"', rerunCellRangerBam = 'f"{config[\'resultDir\']}step3_rerunCellRanger/test/outs/possorted_genome_bam.bam"',windowSize=500))
parseIllumina.setShell("""
gzip -d -c {params.useBarcodeGz} > {params.useBarcode} &&
python ./snuupy/snuupy.py parseIllumina --bam {params.rerunCellRangerBam} --barcode {params.useBarcode} --genome {params.genomeFai} --window {params.windowSize} --parsed {output.parseIlluminaResults}
""")
parseIllumina.generateContent()

In [38]:
snakeAll = jps.SnakeAll(snakeFile)
snakeAll.generateContent(parseIlluminaResults=0)

In [39]:
snakeFile.generateContent(snakePath)

configfile: "/public/home/liuzj/publicPipeline/snuupy/snakemake/config.yaml"
pipelineDir = config['pipelineDir']


rule all:
    input:
        parseIlluminaResults = f"{config['resultDir']}step4_parseIllumina/parseIlluminaResults.index"

rule runCellRanger:
    input:
        firstRunCellRangerConfig = config['firstRunCellRangerConfig']
    output:
        runCellRangerFinished = f"{config['resultDir']}step1_runCellRanger/runCellRangerFinished.empty"
    params:
        gpu = "0"

    threads:2
    shell:
        """
cd {pipelineDir}
python ./optimizedCellRanger/optimizedCellRanger.py runCellRanger -p {input.firstRunCellRangerConfig} &&touch {output.runCellRangerFinished}
        """

rule extractExonBases:
    input:
        runCellRangerFinished = f"{config['resultDir']}step1_runCellRanger/runCellRangerFinished.empty"
    output:
        extractExonBasesFinished = f"{config['resultDir']}step2_extractExonBases/extractExonBasesFinished.empty"
    params:
        extractExonBasesTemp =